## Prepare
import and set variable

In [1]:
# -*- coding:UTF-8 -*-

    
from corpus import *
from vsm import *

c.words

# train the model and create a TfViewer object
tf = TF(c, context_type)
tf.train()
tf_v = TfViewer(c, tf)

# print the most frequent terms in the document
# remember that IPython automatically prints the last cell of a document
tf_v.coll_freqs()


# replace 'topic_range[0]' with a specific number, if you like
k = topic_range[4]

# load the topic model
v = lda_v[k]

#v.topics()

#v.doc_topics(v.labels[:3])

v.aggregate_doc_topics(v.labels[:], normed_sum=True)

#v.dist(v.labels[0], v.labels[1])
%matplotlib inline
import matplotlib.pyplot as plt
from numpy.random import randn
import matplotlib.font_manager as fm
myfont = fm.FontProperties(fname='/home/hongliang/Downloads/msyh.ttf/msyh.ttf')
import sys
reload(sys)
kongzi = []
mengzi = []
xunzi =[]

count = 0
for label in v.labels[:]:
    if '孟子' in label:
        mengzi.append(count)
    elif '论语' in label:
        kongzi.append(count)
    elif '荀子' in label:
        xunzi.append(count)
    count += 1



Running from notebook, using serial load function.
[20, 40, 60, 80, 100]
/home/inpho/inpho/kmx/models/kmx-freq5-nltk-en-freq5-N1000000-LDA-K{0}-document-2000.npz
Loading LDA data from /home/inpho/inpho/kmx/models/kmx-freq5-nltk-en-freq5-N1000000-LDA-K100-document-2000.npz


## functions:

In [2]:
# get term frequency in a particular corpus

def term_freq_labels(term,labels):
    count = 0
    for i in labels: 
        index = c.meta_int('document', {'document_label' : v.labels[i]})
        document = c.view_contexts('document', as_strings=True)[index]
        count += (document == term.decode('utf-8')).sum()
    return count
#get the whole occurrances in a corpus

def allfreq(labels):
    countall = 0    
    for i in labels:
        #wenzi = []
        num = []
        index = c.meta_int('document', {'document_label' : v.labels[i]})
        document = c.view_contexts('document', as_strings=True)[index]
        for a in set(document):
            count = 0
            count += (document == a.decode('utf-8')).sum()
            num.append(count)
            #wenzi.append([a,count])
        countall += sum(num)
    
    return countall



## Get list for topics and topics_weight

In [3]:
#1.找到前十的topic的编号
top10topics = []
top10topicsweight = []
for a in v.aggregate_doc_topics(v.labels[:], normed_sum=True):
    top10topics.append(a[0])
    top10topicsweight.append(a[1])
print top10topics[:10]

#2.生成每个topic的值
wordf = []
wordw = []
word=[]
for a in top10topics[:10]:
    for b in v.topics()[a][:10]:
        wordw.append(b[1])
        word.append(b[0])
        wordf.append(tf_v.coll_freq(b[0]))
def chunks(arr, n):
    return [arr[i:i+n] for i in range(0, len(arr), n)]
#print chunks(wordw,10),chunks(word,10),chunks(wordf,10)


## Figure 2  
Keyword loading in Highest Loading Ten Topics in Our Corpus
Figure 2 y label is word-weight
Figure 2.1 y label is frequency

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
from numpy.random import randn
import matplotlib.font_manager as fm
myfont = fm.FontProperties(fname='/home/hongliang/inpho/kmx/wqy-microhei.ttc')
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

fig2 = plt.figure(figsize=(30,10))
for i in range(0,10):
    ax = plt.subplot(2,5,1+i)
    plt.plot(chunks(wordw,10)[i],'ko--')
    ticks = ax.set_xticks(range(0,10))
    labels = ax.set_xticklabels(chunks(word,10)[i],fontproperties=myfont)
    ax.set_xlabel('characters')
    ax.set_ylabel('weight')
    ax.set_title('Topic:%d'%top10topics[i])
plt.savefig('Figure 2')
    
fig21 = plt.figure(figsize=(30,10))
for i in range(0,10):
    ax = plt.subplot(2,5,1+i)
    plt.plot(chunks(wordf,10)[i],'ko--')
    ticks = ax.set_xticks(range(0,10))
    labels = ax.set_xticklabels(chunks(word,10)[i],fontproperties=myfont)
    ax.set_xlabel('characters')
    ax.set_ylabel('weight')
    ax.set_title('Topic:%d'%top10topics[i])
plt.savefig('Figure 21')

IOError: [Errno 2] No such file or directory: '/home/hongliang/inpho/kmx/wqy-microhei.ttc'

IOError: [Errno 2] No such file or directory: '/home/hongliang/inpho/kmx/wqy-microhei.ttc'

## Figure 3 Highest Loading 10 Topics in Corpus
'Topics','Corpus Weight','Topic Keywords in Descending Order of Weight','Topic Name'

In [ ]:
v.topics(print_len=10)

In [ ]:
from pandas import Series,DataFrame
import pandas as pd
#data = {"Topics":top10topics[:10], "Corpus Weight":top10topicsweight[:10], "Topic Name":[,,,,,,,,,],"Topic Keywords in Descending Order of Weight":chunks(word,10)} 
keywords = []
for a in chunks(word,10):
    keywords.append('  '.join(a))
topicname =['benevolent rule 德政','ethical rulership道德规范','political affairs 政治事件','political affairs 政治事件','治国','cosmos&astrology 宇宙&占卜','ethics&politics 伦理&政治','political affairs 政治事件','nature&cosmos 自然&宇宙','traditional Chinese medicine 中医']  
data3 = {"Topics":top10topics[:10],"Corpus Weight":top10topicsweight[:10],"Topic Keywords in Descending Order of Weight":keywords,"Topic Name":topicname}
f3 = DataFrame(data3,columns=['Topics','Corpus Weight','Topic Keywords in Descending Order of Weight','Topic Name'])
f3

##  Figure 4 Corpus weights Topics 0-99



In [ ]:
#图四，topic weight 分布

fig4 = plt.figure(figsize=(10,5))
#plt.plot(chunks(wordf,10)[i],'ko--')
plt.bar(top10topics, top10topicsweight)
#plt.ylim(0, max(mean_values) * 1.1)
#ticks = ax.set_xticks(range(0,10))
#labels = ax.set_xticklabels(chunks(word,10)[i],fontproperties=myfont)

plt.xlabel('Topic')  
plt.ylabel('Corpus Weight')  
plt.title('Corpus Weights for Topics 0-99')  
plt.savefig('figure4')

## Figure 5 

They are using topic 34. I just copy the term in their topic

need to find our topic again

In [ ]:
term = ['君','人','义','礼','能','贤','莫','天','恶','安']
English = ['nobleman','person','rigteousness','ritual','able','virtuous','do not','day/heaven','evil','peace']

occurrencex = []
occurrencem = []
occurrencek = []

for a in term:
    occurrencek.append(term_freq_labels(a,kongzi))
    occurrencem.append(term_freq_labels(a,mengzi))
    occurrencex.append(term_freq_labels(a,xunzi)) 


    
    
    
k = allfreq(kongzi)
m = allfreq(mengzi)
x = allfreq(xunzi)
k_per_1000 = []
m_per_1000 = []
x_per_1000 = []

for a in occurrencek:
    k_per_1000.append(a*1000/float(k))
for a in occurrencek:
    m_per_1000.append(a*1000/float(m))
for a in occurrencek:
    x_per_1000.append(a*1000/float(x))

Results shows that our numbers are different ， need checked

To do：
Term rank


In [ ]:
data5 = {"Mengzi per 1000 characters":m_per_1000,"Xunzi per 1000 characters":x_per_1000,"Kongzi per 1000 characters":k_per_1000,"Term":term,"English":English,"Occurrences in Kongzi":occurrencek,"Occurrences in Mengzi":occurrencem,"Occurrences in Xunzi":occurrencex}
f5 = DataFrame(data5,columns=['Term','English','Occurrences in Kongzi','Kongzi per 1000 characters','Occurrences in Mengzi','Mengzi per 1000 characters','Occurrences in Xunzi','Xunzi per 1000 characters'])
f5

## Figure 6   Topic 27 Keywords & Weight
Topic 27 没有相同内容的topic（白马非法论和坚白论）

In [ ]:
Chinese = []
Pinyin = ['tian','yu','shi','jian','gui','long','xiong','bing','ji','ren']
English = ['heaven','rain','time','see','turtle','dragon','ominous','illness','propitious','people']
WordWeight = []
for a in v.topics()[17][:10]:
    Chinese.append(a[0])
    WordWeight.append(a[1])

data6 = {"Chinese":Chinese,"pinyin":Pinyin,"English":English,"word weight":WordWeight}
f6 = DataFrame(data6,columns=['Chinese','pinyin','English','word weight'])
f6

## Figure 7 Weightiest 10 topics in each of Analects,Mencius&Xunzi


In [ ]:

la = kongzi

topick=[]

keywordsk = []
keywordk=[]
topicweightk=[]

for a in v.aggregate_doc_topics(v.labels[la[0]:la[-1]+1], normed_sum=True)[:10]:
    topick.append(a[0])
    topicweightk.append(a[1])
    
for a in topick:
    for b in v.topics()[a][:10]:
        keywordsk.append(b[0])

        
for a in chunks(keywordsk,10):
    keywordk.append('  '.join(a))

    


In [ ]:
data71 = {"Topic":topick,"Keywords":keywordk,"Text Weight in Kongzi":topicweightk}
f71 = DataFrame(data71,columns=['Topic','Keywords','Text Weight in Kongzi'])
f71

In [ ]:
la = mengzi

topicm=[]

keywordsm = []
keywordm=[]
topicweightm=[]

for a in v.aggregate_doc_topics(v.labels[la[0]:la[-1]+1], normed_sum=True)[:10]:
    topicm.append(a[0])
    topicweightm.append(a[1])
    
for a in topicm:
    for b in v.topics()[a][:10]:
        keywordsm.append(b[0])

        
for a in chunks(keywordsm,10):
    keywordm.append('  '.join(a))

    

data72 = {"Topic":topicm,"Keywords":keywordm,"Text Weight in Mengzi":topicweightm}
f72 = DataFrame(data72,columns=['Topic','Keywords','Text Weight in Mengzi'])
f72

In [ ]:
la = xunzi

topicx=[]

keywordsx = []
keywordx=[]
topicweightx=[]

for a in v.aggregate_doc_topics(v.labels[la[0]:la[-1]+1], normed_sum=True)[:10]:
    topicx.append(a[0])
    topicweightx.append(a[1])
    
for a in topicx:
    for b in v.topics()[a][:10]:
        keywordsx.append(b[0])

        
for a in chunks(keywordsx,10):
    keywordx.append('  '.join(a))

    
    
data73 = {"Topic":topicx,"Keywords":keywordx,"Text Weight in Xunzi":topicweightx}
f73 = DataFrame(data73,columns=['Topic','Keywords','Text Weight in Xunzi'])
f73

##
Per 1000 Characters = (tf_v.coll_freq(Term[i])  /  sum(text[]))

## Figure 8 Topics Differentiating Analects, Mencius and Xunzi from one another

In [ ]:
kmx=[]
km=[]
kx=[]
dk=[]
mx=[]
dm=[]
dx=[]
for a in range(0,100):
    if a in topick:
        if a in topicm:
            if a in topicx:
                kmx.append(a)
            else:
                km.append(a)
        else:
            if a in topicx:
                kx.append(a)
            else:
                dk.append(a)
    else:
        if a in topicm:
            if a in topicx:
                mx.append(a)
            else:
                dm.append(a)
        else:
            if a in topicx:
                dx.append(a)
print kmx,km,kx,dk,mx,dm,dx
                
        

In [ ]:
Document=[]
TextWeight=[]
TOPIC1=[]
corpusweight =[]
name =['','','','history 历史','politician 政治人物','political affairs 政治事件','law and discipline rite 礼法','politics&war 政治&战争','political affairs 政治事件','public office 官职','political affairs&history 政治事件&历史','sacrifice 祭祀','politics&history 政治&历史','individual morality 个人道德','nature&cosmos 自然&宇宙','government 国家治理','government&ethics 政府&伦理']  
keywords=[]

for a in dk:
    
    i = topick.index(a)
    j = top10topics.index(a)
    Document.append('Analects')
    TextWeight.append(topicweightk[i])
    TOPIC1.append(a)
    corpusweight.append(top10topicsweight[j])
    keywords.append(keywordk[i])
for a in dm:
    
    i = topicm.index(a)
    j = top10topics.index(a)
    Document.append('Mengzi')
    TextWeight.append(topicweightm[i])
    TOPIC1.append(a)
    corpusweight.append(top10topicsweight[j])
    keywords.append(keywordm[i])
for a in dx:
    
    i = topicx.index(a)
    j = top10topics.index(a)
    Document.append('Xunzi')
    TextWeight.append(topicweightx[i])
    TOPIC1.append(a)
    corpusweight.append(top10topicsweight[j])
    keywords.append(keywordx[i])
print len(keywords),len(name)

data8 = {"Document":Document,"Text Weight":TextWeight,"Topic":TOPIC1,"Corpus Weight":corpusweight,"Name":name,"keywords":keywords}
f8 = DataFrame(data8,columns=['Document','Text Weight','Topic','Corpus Weight','Name','keywords'])
f8

## Figure 9 Topic 5's Text Weights

In [ ]:
Text=[]
TextWeight5=[]

#for a in kongzi:
#    Text.append(v.labels[i])
#    for a in 

data9 = {"Text":Text,"Text weight of topic 5":TextWeight5}
f9 = DataFrame(data73,columns=['Text','Text weight of topic 5'])
f9

## Figure 10 Text Weights in A/M/X

In [ ]:
la=kongzi
topick=[]
topicweightk=[]
for a in v.aggregate_doc_topics(v.labels[la[0]:la[-1]+1], normed_sum=True):
    topick.append(a[0])
    topicweightk.append(a[1])
la=mengzi
topicm=[]
topicweightm=[]
for a in v.aggregate_doc_topics(v.labels[la[0]:la[-1]+1], normed_sum=True):
    topicm.append(a[0])
    topicweightm.append(a[1])
la=xunzi
topicx=[]
topicweightx=[]
for a in v.aggregate_doc_topics(v.labels[la[0]:la[-1]+1], normed_sum=True):
    topicx.append(a[0])
    topicweightx.append(a[1])

In [ ]:
fig10 = plt.figure(figsize=(10,5))

ax10_1 = plt.subplot(3,1,1)
plt.bar(topick, topicweightk)
ax10_1.set_xlabel('Topic')
ax10_1.set_ylabel('weight')
ax10_1.set_title('kongzi')

ax10_2 = plt.subplot(3,1,2)
plt.bar(topicm, topicweightm)
ax10_2.set_xlabel('Topic')
ax10_2.set_ylabel('weight')
ax10_2.set_title('mengzi')

ax10_3 = plt.subplot(3,1,3)
plt.bar(topicx, topicweightx)
ax10_3.set_xlabel('Topic')
ax10_3.set_ylabel('weight')
ax10_3.set_title('xunzi')





plt.savefig('Figure 10')

##  Figure 11 Formal Interpretation Matrix of Intersections of Analects, Mencius and Xunzi with Topic Keywords (∩=intersection of sets)


In [ ]:
Document=[]
TextWeight=[]
TOPIC1=[]
corpusweight =[]
name =['ethical rulership 道德规范','benevolent rule 德政','ethics&politics 伦理&政治','kindheartedness 仁','politics&war 政治&战争','filial piety 孝']
keywords=[]

for a in kmx:
    
    i1 = topick.index(a)
    i2 = topicm.index(a)
    i3 = topicx.index(a)
    j = top10topics.index(a)
    Document.append('KMX')
    TextWeight.append(str(topicweightk[i1])+'/'+str(topicweightm[i2])+'/'+str(topicweightx[i3]))
    TOPIC1.append(a)
    corpusweight.append(top10topicsweight[j])

    keywords.append(keywordk[i1])
for a in km:
    
    i1 = topick.index(a)
    i2 = topicm.index(a)
    j = top10topics.index(a)
    Document.append('KM')
    TextWeight.append(str(topicweightk[i1])+'/'+str(topicweightm[i2]))
    TOPIC1.append(a)
    corpusweight.append(top10topicsweight[j])

    keywords.append(keywordm[i2])
for a in kx:
    
    i1 = topick.index(a)
    i3 = topicx.index(a)
    j = top10topics.index(a)
    Document.append('KX')
    TextWeight.append(str(topicweightk[i1])+'/'+str(topicweightx[i3]))
    TOPIC1.append(a)
    corpusweight.append(top10topicsweight[j])

    keywords.append(keywordx[i3])
for a in mx:
    
    i2 = topicm.index(a)
    i3 = topicx.index(a)
    j = top10topics.index(a)
    Document.append('MX')
    TextWeight.append(str(topicweightm[i2])+'/'+str(topicweightx[i3]))
    TOPIC1.append(a)
    corpusweight.append(top10topicsweight[j])

    keywords.append(keywordx[i3])


data11 = {"Document":Document,"Text Weight":TextWeight,"Topic":TOPIC1,"Corpus Weight":corpusweight,"Name":name,"keywords":keywords}
f11 = DataFrame(data11,columns=['Document','Text Weight','Topic','Corpus Weight','Name','keywords'])
f11

In [ ]:
print kongzi

##  nichoal paper
### Figure 3 Highest Loading 10 Topics in Corpus

| Corpus | Topic # Weight | Topic Name  |  Topic Keywords in Descending Order of  Weight |

| 29 | 0.600 | Heaven, Cosmos, & the Way | 天 上 下 大 道 中 人 時 後  地 長 從 成 德 |

some words in nc's paper can't be key in function v.dist_word_top. so need a filter.

In [48]:
list1 = ['天','上','下','大','道','中','人','時','後','地','長','從','成','德']
list = []
for a in list1:
    try:
        v.dist_word_top(a.decode('utf-8'))[0]
        list.append(a.decode('utf-8'))
    except:
        print a



In [36]:
v.dist_word_top(list,[10,9,8,7,6,5,4,3,2,1])[:3]


[LabeledColumn([(u'\u5929',   6.90934509e-02), (u'\u5730',   2.47403122e-02),
        (u'\u4eba',   2.23145355e-02), ..., (u'\u89e3',   1.37752565e-07),
        (u'\u5343',   1.37542258e-07), (u'\u662d',   1.37542258e-07)], 
       dtype=[('word', 'O'), ('value', '<f4')]),
 LabeledColumn([(u'\u9053',   4.83313091e-02), (u'\u4eba',   3.57340351e-02),
        (u'\u5929',   2.98789628e-02), ..., (u'\u5546',   3.54657885e-07),
        (u'\u5f81',   3.54657885e-07), (u'\u5343',   3.54387168e-07)], 
       dtype=[('word', 'O'), ('value', '<f4')]),
 LabeledColumn([(u'\u5929',   5.68127781e-02), (u'\u4eba',   4.64585759e-02),
        (u'\u56fd',   2.38499586e-02), ..., (u'\u7956',   3.87584862e-07),
        (u'\u9c81',   3.86993122e-07), (u'\u81e3',   3.85809670e-07)], 
       dtype=[('word', 'O'), ('value', '<f4')])]

## 尝试使用不同的权值，线性的，或者zipf定律。

In [53]:
from hanziconv import HanziConv

In [56]:
top29 = '天 上 下 大 道 中 人 時 後 地 長 從 成 德'
top97 = '心 見 明 合 失 平 陽 意 神 福 離 陰 各 惑'
top76 = '君 人 公 能 死 見 欲 知 先 得 父 臣 事 辭'
top21 = '民 君 行 國 治 能 得 事 政 下 食 教 官 道'
top23 = '天 道 下 物 知 德 生 能 聖 人 得 身 言 神'
top66 = '國 日 食 歸 成 樂 白 東 亡 師 走 害 望 夜'
top72 = '人 得 相 發 士 小 時 殺 用 意 石 立 莫 主'
top34 = '君 人 義 禮 能 賢 莫 天 惡 安 亂 下 善 性'
top78 = '人 知 言 名 用 治 能 欲 學 文 小 富 彼 盜'
top10 = '今 心 後 力 憂 豈 朝 死 誠 棄 觀 入 罪 古'
x = [top29,top97,top76,top21,top23,top66,top72,top34,top78,top10]
nc_top10 = []
for a in x:
    a = HanziConv.toSimplified(a)
    a = a.split(' ')
    tmp = []
    for b in a:
        try:
            
            v.dist_word_top(b.decode('utf-8'))
            tmp.append(b.decode('utf-8'))
        except:
            print b,a
        
    nc_top10.append(tmp)
    
    

In [61]:
for a in nc_top10:
    print len(a)

## in liner

In [66]:
print 'daoxu'
for a in nc_top10:
    print v.dist_word_top(a,[10,9,8,7,6,5,4,3,2,1])[:1]
    print 'index',nc_top10.index(a)

In [68]:
print 'equal'
for a in nc_top10:
    print v.dist_word_top(a,[1,1,1,1,1,1,1,1,1,1])[:1]
    print 'index',nc_top10.index(a)

### zipf

In [67]:
print 'zipf'
for a in nc_top10:
    print v.dist_word_top(a,[512,256,128,64,32,16,8,4,2,1])[:1]
    print 'index',nc_top10.index(a)

In [ ]:
daoxu

Topic 81     天            地            人            index 0
             德            成            时            
             物            道            生            
             正            

Topic 50     阴            阳            位            index 1
             气            象            起            
             降            始            二            
             易            

Topic 82     君            臣            人            index 2
             国            对            寡            
             民            行            闻            
             事            

Topic 82     君            臣            人            index 3
             国            对            寡            
             民            行            闻            
             事            

Topic 99     人            物            圣            index 4
             道            神            生            
             精            心            可            
             一            

Topic 66     十            日            月            index 5
             二            一            度            
             分            百            岁            

Topic 87     人            二            士            index 6
             掌            十            国            
             事            王            徒            
             史            

Topic 97     人            天            礼            index 7
             能            义            可            
             乱            法            莫            
             道            

Topic 10     人            文            能            index 8
             论            世            书            
             实            事            时            
             圣            

Topic 51     后            太            人            index 9
             帝            皇            王            
             女            长            宫            
             光            
----------------------------------------------------


zipf

Topic 81     天            地            人            index 0
             德            成            时            
             物            道            生            
             正            

Topic 50     阴            阳            位            index 1
             气            象            起            
             降            始            二            
             易            

Topic 82     君            臣            人            index 2
             国            对            寡            
             民            行            闻            
             事            

Topic 82     君            臣            人            index 3
             国            对            寡            
             民            行            闻            
             事            

Topic 99     人            物            圣            index 4
             道            神            生            
             精            心            可            
             一            

Topic 66     十            日            月            index 5
             二            一            度            
             分            百            岁            
             余            

Topic 87     人            二            士            index 6
             掌            十            国            
             事            王            徒            
             史            

Topic 97     人            天            礼            index 7
             能            义            可            
             乱            法            莫            
             道            

Topic 97     人            天            礼            index 8
             能            义            可            
             乱            法            莫            
             道            

Topic 51     后            太            人            index 9
             帝            皇            王            
             女            长            宫            
             光            
----------------------------------------------------


equal

Topic 81     天            地            人            index 0
             德            成            时            
             物            道            生            
             正            

Topic 50     阴            阳            位            index 1
             气            象            起            
             降            始            二            
             易            

Topic 82     君            臣            人            index 2
             国            对            寡            
             民            行            闻            
             事            

Topic 8     民           国           主           index 3
            臣           法           君           
            人           上           治           
            明           

Topic 99     人            物            圣            index 4
             道            神            生            
             精            心            可            
             一            

Topic 83     星            月            日            index 5
             出            天            东            
             北            太            白            
             行            

Topic 87     人            二            士            index 6
             掌            十            国            
             事            王            徒            
             史            

Topic 97     人            天            礼            index 7
             能            义            可            
             乱            法            莫            
             道            

Topic 10     人            文            能            index 8
             论            世            书            
             实            事            时            
             圣            

Topic 51     后            太            人            index 9
             帝            皇            王            
             女            长            宫            
             光            
----------------------------------------------------


